In [1]:
import json
import pickle

In [2]:
data_file = '/Users/sarabeery/Documents/CameraTrapClass/DataWrangling/Databases/ECCV.json'
with open(data_file, 'r') as f:
     data = json.load(f)

In [3]:
tax_file = '/Users/sarabeery/Documents/CameraTrapClass/DataWrangling/CreatingDatabases/category_data_with_taxonomy.json'
with open(tax_file, 'r') as f:
     taxonomy = json.load(f)

In [4]:
db_file = '/Users/sarabeery/Documents/CameraTrapClass/DataWrangling/CreatingDatabases/inat_category_lookup.p'
inat_cat_lookup = pickle.load(open(db_file,'rb'))

In [22]:
images = data['images']
categories = data['categories']
annotations = data['annotations']
cat_names = [[cat['name'],cat['id']] for cat in categories]
for cat in cat_names:
    if cat[0] not in inat_cat_lookup:
        print(cat)
cat_to_id = {}
id_to_cat = {}
for cat in categories:
    cat_to_id[cat['name']] = cat['id']
    id_to_cat[cat['id']] = cat['name']

[u'car', 33]
[u'empty', 30]


In [6]:
print(len(taxonomy))
print(taxonomy[0])

5089
{u'supercategory': u'Insecta', u'ancestors': [{u'name': u'Animalia', u'rank': u'kingdom'}, {u'name': u'Arthropoda', u'rank': u'phylum'}, {u'name': u'Hexapoda', u'rank': u'subphylum'}, {u'name': u'Insecta', u'rank': u'class'}, {u'name': u'Pterygota', u'rank': u'subclass'}, {u'name': u'Coleoptera', u'rank': u'order'}, {u'name': u'Polyphaga', u'rank': u'suborder'}, {u'name': u'Staphyliniformia', u'rank': u'suborder'}, {u'name': u'Staphylinoidea', u'rank': u'superfamily'}, {u'name': u'Silphidae', u'rank': u'family'}, {u'name': u'Nicrophorinae', u'rank': u'subfamily'}, {u'name': u'Nicrophorus', u'rank': u'genus'}], u'id': 1916, u'name': u'Nicrophorus tomentosus'}


In [28]:
for taxa in taxonomy:
    if taxa['name'] == 'Tamias dorsalis':
        print(taxa)

{u'supercategory': u'Mammalia', u'ancestors': [{u'name': u'Animalia', u'rank': u'kingdom'}, {u'name': u'Chordata', u'rank': u'phylum'}, {u'name': u'Vertebrata', u'rank': u'subphylum'}, {u'name': u'Mammalia', u'rank': u'class'}, {u'name': u'Rodentia', u'rank': u'order'}, {u'name': u'Sciuridae', u'rank': u'family'}, {u'name': u'Xerinae', u'rank': u'subfamily'}, {u'name': u'Marmotini', u'rank': u'tribe'}, {u'name': u'Tamias', u'rank': u'genus'}], u'id': 230, u'name': u'Tamias dorsalis'}


In [9]:
print([(cat['name'],inat_cat_lookup[cat['name']]) for cat in categories if cat['name'] in inat_cat_lookup])

[(u'opossum', 'Didelphis'), (u'deer', 'Cervidae'), (u'raccoon', 'Procyon lotor'), (u'squirrel', 'Sciuridae'), (u'bobcat', 'Lynx rufus'), (u'skunk', 'Mephitidae'), (u'dog', 'Canis familiaris'), (u'coyote', 'Canis latrans'), (u'rabbit', 'Leporidae'), (u'bird', 'Aves'), (u'rodent', 'Muridae'), (u'cat', 'Felis catus'), (u'fox', 'Vulpes'), (u'badger', 'Mustelidae')]


In [18]:
map_inat_to_eccv = {}
for taxa in taxonomy:
    #raccoon
    if taxa['name'] == 'Procyon lotor':
        map_inat_to_eccv[taxa['id']] = cat_to_id['raccoon']
    #bobcat
    elif taxa['name'] == 'Lynx rufus':
        map_inat_to_eccv[taxa['id']] = cat_to_id['bobcat']
    #coyote
    elif taxa['name'] == 'Canis latrans':
        map_inat_to_eccv[taxa['id']] = cat_to_id['coyote']
    #cat
    elif taxa['name'] == 'Felis catus':
        map_inat_to_eccv[taxa['id']] = cat_to_id['cat']
    #dog
    elif taxa['name'] == 'Canis familiaris':
        map_inat_to_eccv[taxa['id']] = cat_to_id['dog']
    else: #no direct species match
        for a in taxa['ancestors']:
            #opossum
            if a['rank'] == 'genus' and a['name'] == 'Didelphis':
                map_inat_to_eccv[taxa['id']] = cat_to_id['opossum']
            #fox
            elif a['rank'] == 'genus' and a['name'] == 'Vulpes':
                map_inat_to_eccv[taxa['id']] = cat_to_id['fox']
            #badger
            elif a['rank'] == 'family' and a['name'] == 'Mustelidae':
                map_inat_to_eccv[taxa['id']] = cat_to_id['badger']
            #deer
            elif a['rank'] == 'family' and a['name'] == 'Cervidae':
                map_inat_to_eccv[taxa['id']] = cat_to_id['deer']  
            #squirrel
            elif a['rank'] == 'family' and a['name'] == 'Sciuridae':
                map_inat_to_eccv[taxa['id']] = cat_to_id['squirrel']
            #skunk
            elif a['rank'] == 'family' and a['name'] == 'Mephitidae':
                map_inat_to_eccv[taxa['id']] = cat_to_id['skunk']   
            #rabbit
            elif a['rank'] == 'family' and a['name'] == 'Leporidae':
                map_inat_to_eccv[taxa['id']] = cat_to_id['rabbit']
            #rodent
            elif a['rank'] == 'family' and a['name'] == 'Muridae':
                map_inat_to_eccv[taxa['id']] = cat_to_id['rodent']  
            #bird
            elif a['rank'] == 'class' and a['name'] == 'Aves':
                map_inat_to_eccv[taxa['id']] = cat_to_id['bird']

In [19]:
pickle.dump(map_inat_to_eccv,open('map_inat_to_eccv.p','wb'))

In [20]:
print(len(map_inat_to_eccv))

1042


In [29]:
id_to_cat[map_inat_to_eccv[230]]

u'squirrel'